In [112]:
import requests
from pprint import pprint
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse as req

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep

data_list = list()
df = pd.read_csv('data/table_matches_new.csv')
df = df.drop(columns=['Unnamed: 0'])
count = 1736824
        
while count != 1737122 :
    count += 1
    if (count % 2) == 0:
        sleep(1)
    sleep(1)
    
    url_1 = f"https://soccer365.ru/games/{count}/"
    url_2 = f"https://soccer365.ru/games/{count}/&tab=form_teams"
    response_1 = requests.get(url_1)
    response_2 = requests.get(url_2)

    soup_1 = BeautifulSoup(response_1.text,'lxml')
    soup_2 = BeautifulSoup(response_2.text,'lxml')
    
    title = ['id',"общая информация","Дата",'Команда1',"счет1",'Команда2',"счет2",'Температура',"Признак погоды","поб1","X","поб2","ТМ 2.5","ТБ 2.5"]
    title_Summary_statistics = ['Матчи 1','Матчи 2','Отдых: дни (часы) 1','Отдых: дни (часы) 2','Победы 1','Победы 2','Ничьи 1','Ничьи 2','Поражения 1','Поражения 2','Забито голов 1','Забито голов 2','Пропущено голов 1','Пропущено голов 2','Забито голов за игру 1','Забито голов за игру 2','Пропущено голов за игру 1','Пропущено голов за игру 2','Сухие матчи 1','Сухие матчи 2','обе забьют 1','обе забьют 2','Тотал больше 2.5 1','Тотал больше 2.5 2','Тотал меньше 2.5 1','Тотал меньше 2.5 2','Удары 1','Удары 2','Удары  (соперник) 1','Удары  (соперник) 2','Удары в створ 1','Удары в створ 2','Удары в створ  (соперник) 1','Удары в створ  (соперник) 2','Владение 1','Владение 2','Владение  (соперник) 1','Владение  (соперник) 2','Угловые 1','Угловые 2','Угловые  (соперник) 1','Угловые  (соперник) 2','Нарушения 1','Нарушения 2','Нарушения  (соперник) 1','Нарушения  (соперник) 2','Офсайды 1','Офсайды 2','Офсайды  (соперник) 1','Офсайды  (соперник) 2','Желтые карточки 1','Желтые карточки 2','Желтые карточки  (соперник) 1','Желтые карточки  (соперник) 2','Красные карточки 1','Красные карточки 2','Красные карточки  (соперник) 1','Красные карточки  (соперник) 2']
    full_title = title + title_Summary_statistics
    
    lister = list()
    lister.append(count)
    
    #Общая инфа и время матча
    data = soup_1.find("div",class_="block_header bkcenter").text.replace("\n","")
    date_list = data.split()
    date_time = date_list[-2] + ' ' + date_list[-1]
    lister.append(data)
    lister.append(date_time)

    #Название команд и счет

    result_match_1= soup_1.find("div",class_="live_game left")
    result_match_2= soup_1.find("div",class_="live_game right")

    name_1_com = result_match_1.find("div",class_="live_game_ht").text.replace("\n","")
    goal_1_com = result_match_1.find("div",class_="live_game_goal").text.replace("\n","")

    name_2_com = result_match_2.find("div",class_="live_game_at").text.replace("\n","")
    goal_2_com = result_match_2.find("div",class_="live_game_goal").text.replace("\n","")
    lister.append(name_1_com)
    lister.append(goal_1_com)
    lister.append(name_2_com)
    lister.append(goal_2_com)


    #Погода
    preview_st = soup_1.find("div",class_="preview_item st")
    temper = preview_st.find("span",class_="red").text

    priznac_pog = preview_st.find_all("span",class_="min_gray")[1].text
    lister.append(temper)
    lister.append(priznac_pog)


    #Коэфиценты
    coeficent_all = soup_1.find_all("div",class_="block_body_nopadding")[3]
    coficient = coeficent_all.find_all('div',class_='odds_item odds_logo')

    def cof_func(coficient):
        list_coef = list()
        for i in coficient:
            odds_coeff = i.find_all('div',class_='odds_coeff')
            for j in odds_coeff:
                if j == '\n':
                    continue
                if j == '\xa0':
                    list_coef.clear()
                    break
                list_coef.append(float(j.text))
            if len(list_coef) == 5:
                return  list_coef  
            
    coficient_result = cof_func(coficient)
    lister.extend(coficient_result)
    result_matches_to10 = soup_2.find_all("div", class_="block_body_nopadding")
    Summary_statistics = result_matches_to10[3] 
    table_tr_1 = Summary_statistics.find_all("td")
    Summary_statistics2 = result_matches_to10[4] 
    table_tr_2 = Summary_statistics2.find_all("td")
    
    def reformed_tabl_1(table):
        lens = len(table)
        col_priz = lens / 3
        lister_pr = list()
        lister_res = list()
        for i in range(0,lens):
            if i in [1,4,7,10,13,16,19,22,25,28,31,34,37]:
                lister_pr.append(table[i].text + "1")
                lister_pr.append(table[i].text + "2")
            else:
                lister_res.append(table[i].text.replace("\xa0"," "))
        return(lister_pr,lister_res)

    test_tr1,test_res1 = reformed_tabl_1(table_tr_1)

    #display(test_tr)
    #display(test_res)

    def reformed_tabl_2(table):
        lens = len(table)
        col_priz = lens / 3
        lister_pr = list()
        lister_res = list()
        for i in range(0,lens):
            if i in [1,4,7,10,13,16,19,22,25,28,31,34,37,40,43,46]:
                lister_pr.append(table[i].text + "1")
                lister_pr.append(table[i].text + "2")
            else:
                lister_res.append(table[i].text.replace("\xa0"," "))
        return(lister_pr,lister_res)

    test_tr2,test_res2 = reformed_tabl_2(table_tr_2)

    #display(test_tr)
    #display(test_res)
    final_test_tr = test_tr1 + test_tr2
    final_test_res = test_res1 + test_res2
    lister.extend(final_test_res)
    data_list.append(lister)
    

IndexError: list index out of range

In [ ]:
res_df = pd.DataFrame(
    data = data_list,
    columns= full_title,
)

In [ ]:
res_df.info()
#res_df.to_csv('data/table_res_math_bundes.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 72 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              298 non-null    int64  
 1   общая информация                298 non-null    object 
 2   Дата                            298 non-null    object 
 3   Команда1                        298 non-null    object 
 4   счет1                           298 non-null    object 
 5   Команда2                        298 non-null    object 
 6   счет2                           298 non-null    object 
 7   Температура                     298 non-null    object 
 8   Признак погоды                  298 non-null    object 
 9   поб1                            298 non-null    float64
 10  X                               298 non-null    float64
 11  поб2                            298 non-null    float64
 12  ТМ 2.5                          298 